<a href="https://colab.research.google.com/github/PhucTran1233/Project_celeb-vn/blob/main/Celeb_VN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# liên kết Google drive với colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Tải thư viện cần thiết
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array,array_to_img,ImageDataGenerator, image
from keras.layers import Dense,Activation,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.models import load_model
from keras.models import  Sequential
from keras import regularizers
from keras import layers

In [3]:
train_data = '/content/drive/MyDrive/AI/BaiTapCuoiKi_Celeb_VN/Train'
val_data  =  '/content/drive/MyDrive/AI/BaiTapCuoiKi_Celeb_VN/Validation'

In [4]:
#preprocessing
train_scale = ImageDataGenerator(rescale=1./255,
                                   rotation_range=0.2,
                                   width_shift_range=0.2,   
                                   height_shift_range=0.2,
                                   shear_range=0.3,
                                   zoom_range=0.5,
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   validation_split=0.2)

train_input = train_scale.flow_from_directory(train_data,
                        target_size=(128, 128),
                        batch_size=64,
                        class_mode="categorical")

val_scale = ImageDataGenerator(rescale=1./255)

val_input = val_scale.flow_from_directory(val_data,
                        target_size=(128, 128),
                        batch_size=64,
                        class_mode="categorical")

Found 1904 images belonging to 49 classes.
Found 367 images belonging to 49 classes.


In [5]:
print("match class: ", train_input.class_indices)
print("Tổng cộng: ",train_input.num_classes)

match class:  {'1-Ribi Sachi Thuỷ': 0, '10-Quang Thắng': 1, '11-Vũ Cát Tường': 2, '12-Ánh Viên': 3, '13-NSUT Hồng Vân': 4, '14-Phạm Tuân': 5, '15-Misthy': 6, '16-Phạm Nhật Vượng': 7, '17-Vũ': 8, '18-Karik': 9, '19-Nhật Anh Trắng': 10, '2-Gino Tống': 11, '20-NSUT Tự Long': 12, '21-Sỹ Luân': 13, '22-MC Đại Nghĩa': 14, '23-Vinh Râu': 15, '24-Việt Hương': 16, '25-Trương Minh Quốc Thái': 17, '26-Sơn Tùng MTP': 18, '27-ViruSs': 19, '28-Nguyễn Thị Kim Ngân': 20, '29-Thái Vũ': 21, '3-Nguyễn Huy Hoàng': 22, '30-Chí Tài': 23, '31-Trường Giang': 24, '32-NSUT Thanh Điền': 25, '33-Huỳnh Phương': 26, '34-Lí Hải': 27, '35-Mạc Văn khoa': 28, '36-Phan Mạnh Quỳnh': 29, '37-Xuân Hinh': 30, '38-Minh Nhí': 31, '39-Quang Hải': 32, '4-Đàm Vĩnh Hưng': 33, '40-Trần Tiger': 34, '41-Nguyễn Tử Quảng': 35, '42-Lê Quang Liêm': 36, '43-Tiên Tiên': 37, '44-MC Lại Văn Sâm': 38, '45-Thành Lộc': 39, '46-Đen Vâu': 40, '47-Vân 

In [9]:
#MODEL CNN
model=Sequential()
#TRÍCH XUẤT TẬP DỮ LIỆU(FEATURE EXTRACTORS)
#LAYER 0
model.add(Conv2D(64,(5,5),activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=(128,128,3)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
#LAYER 1
model.add(Conv2D(128,(5,5),activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
#LAYER 2
model.add(Conv2D(256,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
#LAYER 3
model.add(Conv2D(512,(3,3),activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

#FULLY CONNECTED LAYER
#Flatten Layer is used to change the dimension of output from convolution layer, which has 3D, to 2D output
model.add(Flatten())

#Dense
model.add(Dense(4096,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(2048,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(49,activation='softmax'))  
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 128, 128, 64)      4864      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 64, 64, 64)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 64, 64, 128)       204928    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                             
                                                                 
 dropout_15 (Dropout)        (None, 32, 32, 128)      

In [13]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("Best_facecelb_model_val.h5",monitor="val_accuracy",save_best_only=True,mode="max", verbose=1)

In [15]:
face_celeb_train = model.fit(train_input,batch_size=64,epochs=200,verbose=1,validation_data=val_input,callbacks=ckpoint)

Epoch 1/200
30/30 [==============================] - ETA: 0s - loss: 9.8403 - accuracy: 0.0331
Epoch 1: val_accuracy improved from -inf to 0.01907, saving model to Best_facecelb_model_val.h5
30/30 [==============================] - 371s 12s/step - loss: 9.8403 - accuracy: 0.0331 - val_loss: 8.2251 - val_accuracy: 0.0191
Epoch 2/200
30/30 [==============================] - ETA: 0s - loss: 7.2259 - accuracy: 0.0341
Epoch 2: val_accuracy did not improve from 0.01907
30/30 [==============================] - 14s 446ms/step - loss: 7.2259 - accuracy: 0.0341 - val_loss: 7.0028 - val_accuracy: 0.0136
Epoch 3/200
30/30 [==============================] - ETA: 0s - loss: 6.0962 - accuracy: 0.0399
Epoch 3: val_accuracy did not improve from 0.01907
30/30 [==============================] - 13s 445ms/step - loss: 6.0962 - accuracy: 0.0399 - val_loss: 6.2545 - val_accuracy: 0.0191
Epoch 4/200
30/30 [==============================] - ETA: 0s - loss: 5.4260 - accuracy: 0.0415
Epoch 4: val_accuracy did n

In [16]:
# Saving model in JSON file 
from tensorflow.keras.models import model_from_json
model_in_json = model.to_json()
with open('model.json','w') as json_file:
  json_file.write(model_in_json)

In [18]:
# Loading model from JSON file
model_file = open('model.json','r')
json_model=model_file.read()
model2=model_from_json(json_model)
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 128, 128, 64)      4864      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 dropout_14 (Dropout)        (None, 64, 64, 64)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 64, 64, 128)       204928    
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                             
                                                                 
 dropout_15 (Dropout)        (None, 32, 32, 128)      